# XGBoost Model on used m01.1 CNN Model, Original Dataset with no Metadata, All Batch Size, and 100 Epochs

In [ ]:
# debugging
# classes_weights = class_weight.compute_sample_weight(
#     class_weight='balanced',
#     y=y_train
# )
# un_list = []
# un_labels = []
# for i, a_label in enumerate(y_train):
#     if a_label not in un_labels:
#         un_labels.append(a_label)
#         un_list.append(classes_weights[i])
# numToClassName = {value: key for key, value in classNameToNum.items()}
# for i in range(len(un_labels)):
#     print(f"{numToClassName[un_labels[i]]:<25}{un_list[i]:<25}")

00. selfies              0.3220071816091017       
50. fTxtMssgs            0.9869636293589062       
70. eGreetingAndMisc     1.1198180636777129       
81. academicPhotos       2.461296772350757        
10. fmemes               0.4838910585292391       
20. ememes               1.1225633008834885       
30. fSocialMedia         5.572427983539095        
82. academicDigital      7.732664382444118        
40. eSocialMedia         3.1665330393532436       


In [ ]:
# debugging:

def custom_eval_f1_avg(y_pred, y_true):
    global f1_scores
    y_true = y_true.get_label()
    f1 = f1_score(y_true, np.argmax(y_pred, axis=1), average=None)
    f1_avg = f1_score(y_true, np.argmax(y_pred, axis=1), average='macro')
    # appends train f1-score, then val f1-score, and so forth
    # we're storing it in global variable "f1_scores", as xgboost library doesn't accept custom eval function which returns a list (like f1-score metric)
    # you just have to initialize a variable "f1_scores" as empty list before calling any xgb.train() in this notebook
    f1_scores.append(f1)
    return 'f1_score', f1_avg

# read data
X_train = imgTrainFeatures 
y_train = [classNameToNum[className] for className in train_data['label']]
X_val = imgValFeatures
y_val = [classNameToNum[className] for className in val_data['label']]
X_test = imgTestFeatures
y_test = [classNameToNum[className] for className in test_data['label']]
# create model instance
eval_set = [(X_train, y_train), (X_test, y_test)]

# mlogloss <==> cross-entropy loss
# merror <==> 1 - accuracy (i.e., ratio of misclassified labels over all #classifications)
# Side-note: if you'll pass a custom function to eval_metric, then it should return a tuple of string (e.g., 'f1_score_avg') and value (single value, not a list of values).
# Also, if you pass a custom function, then you can't pass any other metrics (i.e., eval_metric=func_name , NOT eval_metric=[func_name, 'mlogloss', ...])
# bst = XGBClassifier(n_estimators=10, eval_metric=["mlogloss", "merror"], objective='multi:softprob', random_state=42)

dtrain = xgb.DMatrix(
    data = X_train,
    label = y_train,
    weight = class_weight.compute_sample_weight(class_weight='balanced',y = y_train)
)

dval = xgb.DMatrix(
    data = X_val,
    label = y_val,
    weight = class_weight.compute_sample_weight(class_weight='balanced',y = y_val)
)

dtest = xgb.DMatrix(
    data = X_test,
    label = y_test,
    weight = class_weight.compute_sample_weight(class_weight='balanced',y = y_test)
)

params = {
    'booster' : 'gbtree', 
    'learning_rate' : 0.3, # alternatively, 'eta'
    'max_depth' : 6,
    'min_child_weight' : 1,
    # doesn't work, you have to pass it in xgb.train()
    # 'early_stopping_rounds':3,

    'seed' : 42,
    'verbosity' : 1,
    # by default, uses all available threads (I think)
    # 'nthread' : 4,
    'num_class' : 9,
    'objective' : 'multi:softprob',
    # in 'eval_metric', if early_stopping is applied, then the last metric will be the one considered
    # however, if you pass a custom metric (in xgb.train()), then that will always be chosen by early stopping
    # also, you can't call a custom function in eval_metric's list of metrics; it has to be in xgb.train()
    'eval_metric' : ['merror', 'mlogloss'] 
}

watchlist = [(dtrain, 'train'), (dval, 'val')] # if early_stopping is applied, then the last data set will be the one considered
n_rounds = 100
results = {}
# regarding early stopping (from xgb.train()'s docstring):
# The method returns the model from the last iteration (not the best one). 
# Use custom callback or model slicing if the best model is desired.
f1_scores = []


bst = xgb.train(params, dtrain, n_rounds, evals=watchlist, evals_result=results, custom_metric=custom_eval_f1_avg, early_stopping_rounds=0)

# make predictions
preds = bst.predict(dtest, output_margin=False)

[0]	train-merror:0.13378	train-mlogloss:1.34860	train-f1_score:0.81785	val-merror:0.24383	val-mlogloss:1.47916	val-f1_score:0.72606
[1]	train-merror:0.10826	train-mlogloss:1.04273	train-f1_score:0.86092	val-merror:0.22793	val-mlogloss:1.21828	val-f1_score:0.75284
[2]	train-merror:0.09552	train-mlogloss:0.84429	train-f1_score:0.87976	val-merror:0.21643	val-mlogloss:1.05092	val-f1_score:0.76749
[3]	train-merror:0.08579	train-mlogloss:0.70102	train-f1_score:0.89029	val-merror:0.21233	val-mlogloss:0.93309	val-f1_score:0.77259
[4]	train-merror:0.07913	train-mlogloss:0.59435	train-f1_score:0.89850	val-merror:0.20817	val-mlogloss:0.84649	val-f1_score:0.77803
[5]	train-merror:0.07153	train-mlogloss:0.50963	train-f1_score:0.90834	val-merror:0.19882	val-mlogloss:0.77721	val-f1_score:0.78865
[6]	train-merror:0.06583	train-mlogloss:0.44255	train-f1_score:0.91478	val-merror:0.19624	val-mlogloss:0.72460	val-f1_score:0.79110
[7]	train-merror:0.06128	train-mlogloss:0.38904	train-f1_score:0.91982	val-m

In [ ]:
# debugging:
# f1_scores[1::2]

[array([1., 1., 1.]),
 array([1., 1., 1.]),
 array([1., 1., 1.]),
 array([1., 1., 1.])]

In [ ]:
# debugging:
epochs_num = len(results["train"]["mlogloss"])
model_version = '04'
results['train']['f1_score'] = f1_scores[0::2]
results['val']['f1_score'] = f1_scores[1::2]
history = {
    'epoch' : [i for i in range(epochs_num)],
    'modelNumber': model_version,
    'loss': np.array(results["train"]["mlogloss"]),
    'acc': np.array(list(map(lambda x : 1 - x, results["train"]["merror"]))).round(5),
    'f1_score': np.array(results["train"]["f1_score"]).round(5),
    'val_loss': np.array(results["val"]["mlogloss"]),
    'val_acc': np.array(list(map(lambda x : 1 - x, results["val"]["merror"]))).round(5),
    'val_f1_score': np.array(results["train"]["f1_score"]).round(5),
}

history.update(getMetrics(y_test, preds, list(classNameToNum.keys())))
print(history.keys())

dict_keys(['epoch', 'modelNumber', 'loss', 'acc', 'f1_score', 'val_loss', 'val_acc', 'val_f1_score', 'test_loss', 'test_acc', 'test_f1_score', 'test_support', 'test_yTrue', 'test_yPred', 'test_confusion_matrix', 'test_confusion_matrix_normalized'])
